In [1]:
import random
import time

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import torchtext

import pandas as pd
from queue import PriorityQueue
import operator

In [2]:
mydevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mydevice

device(type='cpu')

In [3]:
SOS_TOKEN = '<sos>'
EOS_TOKEN = '<eos>'

MAX_LEN = 50

def len_filter(example):
    return len(example.src) <= MAX_LEN and len(example.tgt) <= MAX_LEN

### Load dummy number reversal dataset

In [4]:
# local path of dummy set
train_path = './toy_reversal_export/train/data.txt'
dev_path = './toy_reversal_export/dev/data.txt'

In [5]:
src = torchtext.data.Field(
    batch_first=True, 
    include_lengths=True
    )
tgt = torchtext.data.Field(
    batch_first=True, 
    preprocessing = lambda seq: [SOS_TOKEN] + seq + [EOS_TOKEN]
    )

data_train = torchtext.data.TabularDataset(
        path=train_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )
data_dev = torchtext.data.TabularDataset(
        path=dev_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )

# Loading actual EN-IT Dataset below

In [6]:
# load dataset using pandas
df = pd.read_csv('./ita-eng/ita.txt', sep='\t', header=None)
df.columns = ['ENG', 'IT', 'Source']
display(df.head())

# split data
train = df.sample(frac=0.70)
test = df.sample(frac=0.20)
valid = df.sample(frac=0.10)
print('\n', train.shape, test.shape, valid.shape)

# save data
train.to_csv('train.tsv', sep = '\t', index=False)
test.to_csv('test.tsv', sep = '\t', index=False)
valid.to_csv('valid.tsv', sep = '\t', index=False)

# load paths
train_path = './train.tsv'
test_path = './test.tsv'
valid_path = './valid.tsv'
print(train_path, test_path, valid_path)

# create torchtext objects
src = torchtext.data.Field(batch_first=True, include_lengths=True)
tgt = torchtext.data.Field(batch_first=True, preprocessing = lambda seq: [SOS_TOKEN] + seq + [EOS_TOKEN])

data_train = torchtext.data.TabularDataset(
        path=train_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter)

data_test = torchtext.data.TabularDataset(
        path=test_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter)

data_valid = torchtext.data.TabularDataset(
        path=valid_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter)

,ENG,IT,Source
0,Hi.,Ciao!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
1,Run!,Corri!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
2,Run!,Corra!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,Run!,Correte!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,Who?,Chi?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...



 (239088, 3) (68311, 3) (34155, 3)
./train.tsv ./test.tsv ./valid.tsv


In [7]:
src.build_vocab(data_train)
tgt.build_vocab(data_train)
input_vocab = src.vocab
output_vocab = tgt.vocab

print('20 tokens from input vocab:\n', list(input_vocab.stoi.keys())[:20])
print('\n20 tokens from output vocab:\n', list(output_vocab.stoi.keys())[:20])

print('\nnum training examples:', len(data_train.examples))

item = random.choice(data_train.examples)
print('\nexample train data:')
print('src:\n', item.src)
print('tgt:\n', item.tgt)

#showing output below for toy dataset

20 tokens from input vocab:
 ['<unk>', '<pad>', 'I', 'Tom', 'to', 'you', 'the', 'a', 'is', 'in', 'was', "I'm", 'of', 'have', 'You', 'that', 'do', 'for', 'be', 'He']

20 tokens from output vocab:
 ['<unk>', '<pad>', '<eos>', '<sos>', 'Tom', 'di', 'è', 'a', 'non', 'che', 'Io', 'Non', 'un', 'la', 'il', 'ha', 'per', 'in', 'sono', 'una']

num training examples: 239086

example train data:
src:
 ['Tom', 'said', 'that', 'Mary', 'seemed', 'busy.']
tgt:
 ['<sos>', 'Tom', 'ha', 'detto', 'che', 'Mary', 'sembrava', 'impegnata.', '<eos>']


### Model definition and training functions

In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, myinput, hidden):
        embedded = self.embedding(myinput).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=mydevice)

    
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=mydevice)

In [9]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,
          max_length=MAX_LEN, teacher_forcing_ratio=0.5):
    
    # get an initial hidden state for the encoder
    encoder_hidden = encoder.initHidden()

    # zero the gradients of the optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    # get the seq lengths, used for iterating through encoder/decoder
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    # create empty tensor to fill with encoder outputs
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

    # create a variable for loss
    loss = 0
    
    # pass the inputs through the encoder
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    # create a start-of-sequence tensor for the decoder
    decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

    # set the decoder hidden state to the final encoder hidden state
    decoder_hidden = encoder_hidden

    # decide if we will use teacher forcing
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input
                
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        
        if use_teacher_forcing:
            decoder_input = target_tensor[di]
        
        if decoder_input.item() == output_vocab.stoi[EOS_TOKEN]:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [10]:
def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.01, teacher_forcing_ratio=0.5):
    print(f'Running {n_iters} epochs...')
    print_loss_total = 0
    print_loss_epoch = 0
    
    encoder_optim = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optim = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    # note batch size of 1, just for simplicity
    batch_iterator = torchtext.data.Iterator(
        dataset=data_train, batch_size=1,
        sort=False, sort_within_batch=True,
        sort_key=lambda x: len(x.src),
        device=mydevice, repeat=False)
    
    criterion = nn.NLLLoss()
    
    for e in range(n_iters):
        batch_generator = batch_iterator.__iter__()
        step = 0
        start = time.time()
        for batch in batch_generator:
            step += 1
            
            # get the input and target from the batch iterator
            input_tensor, input_lengths = getattr(batch, 'src')
            target_tensor = getattr(batch, 'tgt')
            
            # this is because we're not actually using the batches.
            # batch size is 1 and this just selects that first one
            input_tensor = input_tensor[0]
            target_tensor = target_tensor[0]
            
            loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optim, decoder_optim, criterion, teacher_forcing_ratio=teacher_forcing_ratio)
            print_loss_total += loss
            print_loss_epoch += loss
            
            
            if step % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                t = (time.time() - start) / 60
                print(f'step: {step}\t avg loss: {print_loss_avg:.2f}\t time for {print_every} steps: {t:.2f} min')
                start = time.time()
        
        print_loss_avg = print_loss_epoch / step
        print_loss_epoch = 0
        print(f'End of epoch {e}, avg loss {print_loss_avg:.2f}')  

###  Create and train a model

In [11]:
hidden_size = 32
encoder1 = EncoderRNN(len(input_vocab), hidden_size).to(mydevice)
decoder1 = DecoderRNN(hidden_size, len(output_vocab)).to(mydevice)

Here are some guidelines for how much training to expect. Note that these *guidelines*; they are not exact.

Only 1 epoch is needed for the number reversal dataset. This produces near-perfect results, and should take less than 5 minutes to run on a CPU.



# Training on EN-IT Dataset

In [12]:
trainIters(encoder1, decoder1, 1, print_every=1000, teacher_forcing_ratio=0.75)

Running 1 epochs...
step: 1000	 avg loss: 6.39	 time for 1000 steps: 1.97 min
step: 2000	 avg loss: 5.57	 time for 1000 steps: 1.95 min
step: 3000	 avg loss: 5.33	 time for 1000 steps: 2.00 min
step: 4000	 avg loss: 5.27	 time for 1000 steps: 2.04 min
step: 5000	 avg loss: 5.13	 time for 1000 steps: 1.92 min
step: 6000	 avg loss: 5.11	 time for 1000 steps: 1.99 min
step: 7000	 avg loss: 5.09	 time for 1000 steps: 1.92 min
step: 8000	 avg loss: 5.08	 time for 1000 steps: 1.94 min
step: 9000	 avg loss: 5.01	 time for 1000 steps: 1.91 min
step: 10000	 avg loss: 4.99	 time for 1000 steps: 1.89 min
step: 11000	 avg loss: 4.93	 time for 1000 steps: 1.92 min
step: 12000	 avg loss: 4.88	 time for 1000 steps: 1.91 min
step: 13000	 avg loss: 4.86	 time for 1000 steps: 1.91 min
step: 14000	 avg loss: 4.91	 time for 1000 steps: 1.91 min
step: 15000	 avg loss: 4.91	 time for 1000 steps: 1.97 min
step: 16000	 avg loss: 4.89	 time for 1000 steps: 2.01 min
step: 17000	 avg loss: 4.84	 time for 1000 st

In [13]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LEN):
    with torch.no_grad():
        input_tensor = torch.tensor([input_vocab.stoi[word] for word in sentence], device=mydevice)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

        decoder_hidden = encoder_hidden

        decoded_words = []
        
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            next_word = output_vocab.itos[topi.item()]
            decoded_words.append(next_word)
            if next_word == EOS_TOKEN:
                break

            decoder_input = topi.squeeze().detach()

        return decoded_words

# Translating some example sentences

In [14]:
# This selects 5 random datapoints from the training data and shows the generated sequence

for i in range(5):
    item = random.choice(data_train.examples)
    seq = item.src
    print(seq)
    words = evaluate(encoder1, decoder1, seq)
    print(' '.join(words))
    print()

['Is', 'it', 'worth', 'fixing?']
<sos> È la <eos>

['Do', 'you', 'think', 'about', 'Tom', 'a', 'lot?']
<sos> Spero che Tom sia un po' <eos>

['Does', 'this', 'book', 'belong', 'to', 'you?']
<sos> Lui ha a <eos>

["She's", 'painting', 'her', 'room', 'white.']
<sos> Ci sono la <eos>

['Tom', "doesn't", 'like', 'green', 'peppers.']
<sos> Tom non piace la mia <eos>



# Beam Search Algorithm

original source:

https://github.com/budzianowski/PyTorch-Beam-Search-Decoding/blob/master/decode_beam.py

In [15]:
SOS_token = 0
EOS_token = 1

class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward
    

def beam_decode(target_tensor, decoder_hidden, beam_width):
    '''
    :param target_tensor: target indexes tensor of shape [B, T] where B is the batch size and T is the maximum length of the output sentence
    :param decoder_hidden: input tensor of shape [1, B, H] for start of the decoding
    :param encoder_outputs: if you are using attention mechanism you can pass encoder outputs, [T, B, H] where T is the maximum length of input sentence
    :return: decoded_batch
    '''

    topk = 1  # how many sentence do you want to generate
    decoded_batch = []
    
    # decoding goes sentence by sentence
    for idx in range(target_tensor.size(0)):
        
        #####################################################################################
        # some changes made here from the original version are as follows:
        # beam_width is now made a user given input instead of a fixed constant
        # unsqueeze part was removed since the original code data format was different
        # also unused encoder_output line was removed since attention is not used
        #####################################################################################
        
        # Start with the start of the sentence token
        decoder_input = torch.LongTensor([[SOS_token]], device=mydevice)

        # Number of sentence to generate
        endnodes = []
        number_required = min((topk + 1), topk - len(endnodes))

        # starting node -  hidden vector, previous node, word id, logp, length
        node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
        nodes = PriorityQueue()

        # start the queue
        nodes.put((-node.eval(), node))
        qsize = 1
        
        # start beam search
        while True:
            # give up when decoding takes too long
            if qsize > 2000: break
            
            # fetch the best node
            score, n = nodes.get()
            decoder_input = n.wordid
            decoder_hidden = n.h
            
            if n.wordid.item() == EOS_token and n.prevNode != None:
                endnodes.append((score, n))
                # if we reached maximum # of sentences required
                if len(endnodes) >= number_required:
                    break
                else:
                    continue

            # decode for one step using decoder
            decoder_output, decoder_hidden = decoder1(decoder_input, decoder_hidden)
            
            # PUT HERE REAL BEAM SEARCH OF TOP
            log_prob, indexes = torch.topk(decoder_output, beam_width)
            
            nextnodes = []
            for new_k in range(beam_width):
                decoded_t = indexes[0][new_k].view(1, -1)
                log_p = log_prob[0][new_k].item()
                
                node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp + log_p, n.leng + 1)
                score = -node.eval()
                nextnodes.append((score, node))
            
            # put them into queue
            for i in range(len(nextnodes)):
                score, nn = nextnodes[i]
                nodes.put((score, nn))
                # increase qsize
            qsize += len(nextnodes) - 1
        
        # choose nbest paths, back trace them
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(topk)]
        
        utterances = []
        for score, n in sorted(endnodes, key=operator.itemgetter(0)):
            utterance = []
            
            #####################################################################################
            # pulling words from output vocabulary
            # this is another change that i made from the original code
            # the original code was adding the word ID but i am pulling decoded word directly from the dictionary
            #####################################################################################
            
            utterance.append(output_vocab.itos[n.wordid])
            
            # back trace
            while n.prevNode != None:
                n = n.prevNode
                # same here as above, pulling words from output vocabulary
                utterance.append(output_vocab.itos[n.wordid])
            
            utterance = utterance[::-1]
            utterances.append(utterance)
            
        decoded_batch.append(utterances)

    return decoded_batch

# Create some evaluation functions which makes use of above "beam_decode" function

In [16]:
def evaluate_beam(encoder, decoder, sentence, max_length=MAX_LEN):
    with torch.no_grad():
        
        # most of this code is unchanged        
        # target sentence in integer encoded format
        input_tensor = torch.tensor([input_vocab.stoi[word] for word in sentence], device=mydevice)
        print(input_tensor)
        input_length = input_tensor.size()[0]
        
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]
                
        decoder_hidden = encoder_hidden
        
        decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        
        #####################################################################################
        # we dont need encoder outputs since we are not using Attention mechanism
        # hence ignored in beam_decode method
        #####################################################################################
        
        # calling the actual function
        decoded_words = beam_decode(input_tensor, decoder_hidden, 10)
        
        return decoded_words

# Generating IT translated Sentences

In [17]:
for i in range(5):
    item = random.choice(data_train.examples)
    seq = item.src
    print(seq)
    
    words = evaluate_beam(encoder1, decoder1, seq)
    print('total decoded words:', len(words))
    
    decoded_sent = []
    for w in words:
        for i in w:
            for k in i:
                if k != '<unk>' and k != '<sos>' and k != '<eos>':
                    decoded_sent.append(k)
    print(decoded_sent)
    print()

["You're", 'not', 'rich.']
tensor([ 41,  30, 674])
total decoded words: 3
['non', 'sei']

['I', 'love', 'music,', 'especially', 'rock.']
tensor([   2,  132, 7440, 5310, 4049])
total decoded words: 5
['chiedo', 'che', 'sia']

['You', 'are', 'not', 'a', 'child', 'any', 'more.']
tensor([ 14,  25,  30,   7, 968, 146, 650])
total decoded words: 7
['non', 'è', 'ancora', 'un', "po'", 'di']

['Tom', 'is', 'former', 'CIA.']
tensor([   3,    8, 5946, 8398])
total decoded words: 4
['Tom', 'è', 'la']

["I'm", 'almost', 'ready', 'to', 'leave.']
tensor([ 11, 290, 286,   4, 464])
total decoded words: 5
['sono', 'stato', 'a']

